# Import

In [ ]:
!pip install catboost pytorch-lightning pytorch-forecasting optuna -q

In [ ]:

import os, gc, math, random
import numpy as np
import pandas as pd
from datetime import timedelta

import torch
import lightning as L
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

from catboost import CatBoostRegressor, Pool

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics import RMSE
from pytorch_forecasting.data import GroupNormalizer

import optuna
import glob, re
from tqdm import tqdm
import logging

logging.getLogger("lightning").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)


# Fixed RandomSeed & Setting Hyperparameter

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

In [ ]:
# 공통 설정
ENC_LEN  = 28
PRED_LEN = 7
ROLL_WINS = [7, 14, 28]

# CatBoost 하이퍼파라미터
CAT_PARAMS = dict(
    depth=8,
    learning_rate=0.05,
    iterations=2000,
    loss_function="RMSE",
    l2_leaf_reg=5,
    random_seed=42,
    verbose=200,
    od_type="Iter",
    od_wait=200,
)

# TFT-mini 하이퍼파라미터 (dropout tuned later)
TFT_PARAMS = dict(
    learning_rate=2e-3,
    hidden_size=64,
    attention_head_size=2,
    hidden_continuous_size=32,
    lstm_layers=1,
)

# 앙상블 가중치
W_CAT = 0.4
W_TFT = 0.6


# Data Load

In [ ]:
BASE_DIR = '/open'
file_path = BASE_DIR
train_path  = os.path.join(BASE_DIR,'train', "train.csv")
sample_path = os.path.join(BASE_DIR, "sample_submission.csv")
test_paths  = [os.path.join(BASE_DIR,'test', f"TEST_0{i}.csv") for i in range(10)]

train  = pd.read_csv(train_path,  parse_dates=["영업일자"])
sample = pd.read_csv(sample_path)
tests  = {f"TEST_0{i}": pd.read_csv(p, parse_dates=["영업일자"]) for i, p in enumerate(test_paths)}

# 라벨 고정 및 키 분해
train["매출수량"] = pd.to_numeric(train["매출수량"], errors="coerce").fillna(0).clip(lower=0)
train["영업장명_메뉴명"] = train["영업장명_메뉴명"].astype(str)
train["업장명"] = train["영업장명_메뉴명"].str.split("_", n=1).str[0]
train["메뉴명"] = train["영업장명_메뉴명"].str.split("_", n=1).str[1].fillna("NA")
train = train.sort_values(["영업장명_메뉴명","영업일자"]).reset_index(drop=True)

# ID 매핑
def make_id_map(s: pd.Series):
    uniq = s.astype(str).unique().tolist()
    return {k: i for i, k in enumerate(uniq)}

store2id = make_id_map(train["업장명"])
item2id  = make_id_map(train["메뉴명"])
pair2id  = make_id_map(train["업장명"] + "###" + train["메뉴명"])

train["store_id"] = train["업장명"].map(store2id).astype(int)
train["item_id"]  = train["메뉴명"].map(item2id).astype(int)
train["pair_id"]  = (train["업장명"] + "###" + train["메뉴명"]).map(pair2id).astype(int)

# 공휴일/이벤트 정의
KOREA_HOLIDAYS = set([
    # ----- 2023 -----
    '2023-01-01',  # 신정
    '2023-01-21',  # 설날 연휴 시작
    '2023-01-22',  # 설날
    '2023-01-23',  # 설날 연휴
    '2023-01-24',  # 대체공휴일
    '2023-03-01',  # 삼일절
    '2023-05-05',  # 어린이날
    '2023-05-27',  # 부처님오신날
    '2023-05-29',  # 부처님오신날 대체휴일
    '2023-06-06',  # 현충일
    '2023-08-15',  # 광복절
    '2023-09-28',  # 추석 연휴 시작
    '2023-09-29',  # 추석
    '2023-09-30',  # 추석 연휴
    '2023-10-02',  # 임시공휴일
    '2023-10-03',  # 개천절
    '2023-10-09',  # 한글날
    '2023-12-25',  # 크리스마스
    # ----- 2024 -----
    '2024-01-01',  # 신정
    '2024-02-09',  # 설날 연휴 시작
    '2024-02-10',  # 설날
    '2024-02-11',  # 설날 연휴
    '2024-02-12',  # 대체공휴일
    '2024-03-01',  # 삼일절
    '2024-04-10',  # 제22대 국회의원 선거
    '2024-05-05',  # 어린이날
    '2024-05-06',  # 어린이날 대체휴일
    '2024-05-15',  # 석가탄신일
    '2024-06-06',  # 현충일
    '2024-08-15',  # 광복절
    '2024-09-16',  # 추석 연휴 시작
    '2024-09-17',  # 추석
    '2024-09-18',  # 추석 연휴
    '2024-10-03',  # 개천절
    '2024-10-09',  # 한글날
    '2024-12-25',  # 크리스마스
])

EVENTS_GLOBAL = set([

])

EVENTS_TARGETED = {

}

def expand_dates(dates, days=3):
    if not dates:
        return pd.DataFrame(columns=["영업일자","near_flag"])
    base = pd.to_datetime(sorted(pd.to_datetime(list(dates)))).normalize()
    offs = pd.to_timedelta(np.arange(-days, days+1), unit="D")
    expanded = (base.values[:, None] + offs.values[None, :]).ravel()
    out = pd.DataFrame({"영업일자": pd.to_datetime(expanded).normalize(), "near_flag": 1})
    return out.drop_duplicates()

df_near_glob = expand_dates(EVENTS_GLOBAL, days=3)
near_glob_map = df_near_glob.set_index("영업일자")["near_flag"] if len(df_near_glob) else pd.Series(dtype="int")

def add_domain_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    dt = pd.to_datetime(df["영업일자"])
    df["dow"] = dt.dt.weekday
    df["month"] = dt.dt.month
    df["is_weekend"] = (df["dow"] >= 5).astype(int)

    df["dow_sin"] = np.sin(2*np.pi*df["dow"]/7.0).astype(np.float32)
    df["dow_cos"] = np.cos(2*np.pi*df["dow"]/7.0).astype(np.float32)
    df["month_sin"] = np.sin(2*np.pi*df["month"]/12.0).astype(np.float32)
    df["month_cos"] = np.cos(2*np.pi*df["month"]/12.0).astype(np.float32)

    df["is_spring"] = df["month"].isin([3,4,5]).astype(int)
    df["is_summer"] = df["month"].isin([6,7,8]).astype(int)
    df["is_fall"]   = df["month"].isin([9,10,11]).astype(int)
    df["is_winter"] = df["month"].isin([12,1,2]).astype(int)

    df["is_peak_summer"] = df["month"].isin([7,8]).astype(int)
    df["is_peak_winter"] = df["month"].isin([12,1,2]).astype(int)

    date_str = dt.dt.strftime("%Y-%m-%d")
    date_key = dt.dt.normalize()
    df["is_holiday"] = date_str.isin(KOREA_HOLIDAYS).astype(int)
    prev_date = (dt - pd.Timedelta(days=1)).dt.strftime("%Y-%m-%d")
    next_date = (dt + pd.Timedelta(days=1)).dt.strftime("%Y-%m-%d")
    df["before_holiday"] = prev_date.isin(KOREA_HOLIDAYS).astype(int)
    df["after_holiday"]  = next_date.isin(KOREA_HOLIDAYS).astype(int)

    if len(KOREA_HOLIDAYS) > 0:
        holi_sorted = pd.to_datetime(sorted(list(KOREA_HOLIDAYS))).normalize()
        df_h = pd.DataFrame({"d": holi_sorted})
        grp = (df_h["d"].diff().dt.days.ne(1)).cumsum()
        runlen = df_h.groupby(grp)["d"].transform("size")
        run_map = pd.Series(runlen.values, index=df_h["d"].values)
        df["is_holiday_run"] = date_key.map(run_map).fillna(0).astype(np.int16)
    else:
        df["is_holiday_run"] = 0

    start_su = pd.to_datetime(dt.dt.year.astype(str) + "-07-15")
    end_su   = pd.to_datetime(dt.dt.year.astype(str) + "-08-31")
    df["is_summer_vac"] = ((dt >= start_su) & (dt <= end_su)).astype(int)

    start_w1 = pd.to_datetime(dt.dt.year.astype(str) + "-12-20")
    end_w1   = pd.to_datetime(dt.dt.year.astype(str) + "-12-31")
    start_w2 = pd.to_datetime((dt.dt.year-1).astype(str) + "-12-20")
    end_w2   = pd.to_datetime(dt.dt.year.astype(str) + "-02-28")
    df["is_winter_vac"] = (((dt >= start_w1) & (dt <= end_w1)) | ((dt >= start_w2) & (dt <= end_w2))).astype(int)

    df["EVENT_SF_SZN"]      = df["month"].isin([3,4,5,9,10,11]).astype(int)
    df["EVENT_SUMMER_SZN"]  = df["month"].isin([6,7,8]).astype(int)
    df["EVENT_WINTER_SZN"]  = df["month"].isin([12,1,2]).astype(int)

    df["is_event_global"] = date_str.isin(EVENTS_GLOBAL).astype(int)
    df["near_event_global"] = 0 if near_glob_map.empty else date_key.map(near_glob_map).fillna(0).astype("int8")

    df["is_event_target"] = 0
    df["near_event_target"] = 0
    if "업장명" in df.columns and "메뉴명" in df.columns:
        for (s_name, i_name), dates in EVENTS_TARGETED.items():
            if not dates:
                continue
            m = (df["업장명"].eq(s_name)) & (df["메뉴명"].eq(i_name))
            df.loc[m, "is_event_target"] = df.loc[m, "영업일자"].dt.strftime("%Y-%m-%d").isin(set(dates)).astype(int)
            df_near = expand_dates(dates, days=3)
            if len(df_near):
                mapper = df_near.set_index("영업일자")["near_flag"]
                df.loc[m, "near_event_target"] = df.loc[m, "영업일자"].dt.normalize().map(mapper).fillna(0).astype("int8")

    return df

train = add_domain_features(train)

# 롤링 통계(과거 기반)
for w in ROLL_WINS:
    train[f"roll_mean_{w}"] = train.groupby("영업장명_메뉴명")["매출수량"].transform(lambda s: s.rolling(w, min_periods=1).mean())

# CatBoost용 피처 구성
cat_features_cols = ["업장명","메뉴명"]
base_num_features = [
    "is_weekend","dow_sin","dow_cos","month_sin","month_cos",
    "is_spring","is_summer","is_fall","is_winter",
    "is_peak_summer","is_peak_winter",
    "is_holiday","before_holiday","after_holiday","is_holiday_run",
    "is_summer_vac","is_winter_vac",
    "EVENT_SF_SZN","EVENT_SUMMER_SZN","EVENT_WINTER_SZN",
    "is_event_global","near_event_global","is_event_target","near_event_target",
] + [f"roll_mean_{w}" for w in ROLL_WINS]

# Define Model

# Train

In [ ]:
# --------------------------------------------------------------
# (1) 미래에 '알고 있는' 달력/이벤트 피처 목록
# --------------------------------------------------------------
known_future_cols = [
    "dow","month","is_weekend","dow_sin","dow_cos","month_sin","month_cos",
    "is_spring","is_summer","is_fall","is_winter",
    "is_peak_summer","is_peak_winter",
    "is_holiday","before_holiday","after_holiday","is_holiday_run",
    "is_summer_vac","is_winter_vac",
    "EVENT_SF_SZN","EVENT_SUMMER_SZN","EVENT_WINTER_SZN",
    "is_event_global","near_event_global","is_event_target","near_event_target",
]

# --------------------------------------------------------------
# (2) CatBoost 학습용: shift 벡터화 + fragmentation 방지
# --------------------------------------------------------------
train_sorted = train.sort_values(["영업장명_메뉴명","영업일자"]).reset_index(drop=True)
g = train_sorted.groupby("영업장명_메뉴명", sort=False)
train_sorted["hist_ok"] = g.cumcount() >= (ENC_LEN - 1)

base_for_shift = ["매출수량"] + known_future_cols
shift_blocks = []
for h in range(1, PRED_LEN + 1):
    block = g[base_for_shift].shift(-h)
    rename_map = {"매출수량": f"y_H{h}"}
    rename_map.update({c: f"{c}_H{h}" for c in known_future_cols})
    block = block.rename(columns=rename_map)
    shift_blocks.append(block)

train_shift = pd.concat(shift_blocks, axis=1)
train_sorted = pd.concat([train_sorted, train_shift], axis=1).copy()

def build_catboost_Xy_by_shift(df: pd.DataFrame, stride: int = 1):
    Xy = {}
    base_cols = cat_features_cols + [f"roll_mean_{w}" for w in ROLL_WINS]
    for c in base_cols:
        if c in cat_features_cols:
            df[c] = df[c].astype("category")
        else:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("float32")
    for h in range(1, PRED_LEN + 1):
        fut_cols_h = [f"{c}_H{h}" for c in known_future_cols]
        target_col = f"y_H{h}"
        mask = df["hist_ok"] & df[target_col].notna()
        use = df.loc[mask, base_cols + fut_cols_h + [target_col]].copy() if stride==1 else (
            df.loc[
                df[mask].groupby("영업장명_메뉴명", sort=False).apply(lambda x: x.iloc[::stride]).reset_index(level=0, drop=True).index,
                base_cols + fut_cols_h + [target_col]
            ].copy()
        )
        for c in fut_cols_h:
            use[c] = pd.to_numeric(use[c], errors="coerce").astype("float32")
        use[target_col] = pd.to_numeric(use[target_col], errors="coerce").astype("float32")
        X = use[base_cols + fut_cols_h].reset_index(drop=True)
        y = use[target_col].reset_index(drop=True)
        Xy[h] = (X, y)
    return Xy

Xy_h = build_catboost_Xy_by_shift(train_sorted, stride=1)

# --------------------------------------------------------------
# (3) TFT용 데이터셋/로더
# --------------------------------------------------------------
train_tft = train.copy()
train_tft["time_idx"] = (train_tft["영업일자"] - train_tft["영업일자"].min()).dt.days.astype(int)
for c in ["store_id", "item_id", "pair_id"]:
    train_tft[c] = train_tft[c].astype(str)

static_categoricals = ["store_id", "item_id", "pair_id"]

time_varying_known_reals = [
    "time_idx",
    "dow_sin","dow_cos","month_sin","month_cos",
    "is_weekend","is_spring","is_summer","is_fall","is_winter",
    "is_peak_summer","is_peak_winter",
    "is_holiday","before_holiday","after_holiday","is_holiday_run",
    "is_summer_vac","is_winter_vac",
    "EVENT_SF_SZN","EVENT_SUMMER_SZN","EVENT_WINTER_SZN",
    "is_event_global","near_event_global","is_event_target","near_event_target",
]

time_varying_unknown_reals = ["매출수량"] + [f"roll_mean_{w}" for w in ROLL_WINS]

for c in time_varying_known_reals:
    if c != "time_idx":
        train_tft[c] = pd.to_numeric(train_tft[c], errors="coerce").astype("float32")
train_tft["time_idx"] = pd.to_numeric(train_tft["time_idx"], errors="coerce").astype("int64")
for c in [f"roll_mean_{w}" for w in ROLL_WINS] + ["매출수량"]:
    train_tft[c] = pd.to_numeric(train_tft[c], errors="coerce").astype("float32")

training_cutoff = train_tft["time_idx"].max() - PRED_LEN

# --- Modified: added GroupNormalizer ---
tft_dataset = TimeSeriesDataSet(
    train_tft[train_tft.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="매출수량",
    group_ids=["pair_id"],
    min_encoder_length=ENC_LEN,
    max_encoder_length=ENC_LEN,
    min_prediction_length=PRED_LEN,
    max_prediction_length=PRED_LEN,
    static_categoricals=static_categoricals,
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_reals=time_varying_unknown_reals,
    target_normalizer=GroupNormalizer(groups=["pair_id"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(tft_dataset, train_tft, predict=True, stop_randomization=True)
train_loader = tft_dataset.to_dataloader(train=True, batch_size=256, num_workers=2)
val_loader   = validation.to_dataloader(train=False, batch_size=256, num_workers=2)

print('[Define Dataset] dataset ready.')


In [ ]:
# ==============================================================
# XGBoost 학습 (h=1..7) -- Added
# ==============================================================
import xgboost as xgb
xgb_models = {}

for h in range(1, PRED_LEN + 1):
    Xh, yh = Xy_h[h]
    if len(Xh) == 0:
        print(f"[XGBoost][H{h}] 학습 데이터가 없습니다. 건너뜁니다.")
        continue
    X_tr, y_tr, X_va, y_va = split_train_valid(Xh, yh, valid_ratio=0.1)
    print(f"[XGBoost][H{h}] train={len(X_tr):,}  valid={len(X_va):,}")
    model = xgb.XGBRegressor(
        max_depth=8,
        learning_rate=0.05,
        n_estimators=2000,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=5,
        objective='reg:squarederror',
        tree_method='hist',
        random_state=42,
        enable_categorical=True,
    )
    model.fit(X_tr, y_tr,
              eval_set=[(X_va, y_va)],
              verbose=200,
              early_stopping_rounds=100)
    xgb_models[h] = model
    gc.collect()


In [ ]:
# ==============================================================
# CatBoost 학습 (h=1..7)
# ==============================================================
if "split_train_valid" not in globals():
    def split_train_valid(X: pd.DataFrame, y: pd.Series, valid_ratio=0.1):
        n = len(X); k = int(n * (1 - valid_ratio))
        return (X.iloc[:k].reset_index(drop=True), y.iloc[:k].reset_index(drop=True),
                X.iloc[k:].reset_index(drop=True), y.iloc[k:].reset_index(drop=True))

if "cat_col_indices" not in globals():
    def cat_col_indices(cols):
        return [i for i, c in enumerate(cols) if c in cat_features_cols]

cat_models = {}

for h in range(1, PRED_LEN + 1):
    Xh, yh = Xy_h[h]
    if len(Xh) == 0:
        print(f"[CatBoost][H{h}] 학습 데이터가 없습니다. 건너뜁니다.")
        continue

    # 범주형 피처 인덱스
    cat_idx = cat_col_indices(Xh.columns)

    # 단순 시계열 분할(뒤쪽 10% 검증)
    X_tr, y_tr, X_va, y_va = split_train_valid(Xh, yh, valid_ratio=0.1)
    print(f"[CatBoost][H{h}] train={len(X_tr):,}  valid={len(X_va):,}")

    train_pool = Pool(X_tr, y_tr, cat_features=cat_idx)
    valid_pool = Pool(X_va, y_va, cat_features=cat_idx)

    # 모델 생성 및 학습
    cb = CatBoostRegressor(**CAT_PARAMS)
    cb.fit(
        train_pool,
        eval_set=valid_pool,
        use_best_model=True,
        verbose=200
    )
    cat_models[h] = cb

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [ ]:
# ==============================================================
# TFT-mini 학습 (dropout 최적화 포함)
# ==============================================================

# --- Modified: dropout optimization with Optuna ---
def tft_objective(trial):
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    model = TemporalFusionTransformer.from_dataset(
        tft_dataset,
        loss=RMSE(),
        log_interval=200,
        reduce_on_plateau_patience=4,
        dropout=dropout,
        weight_decay=1e-2,
        **TFT_PARAMS
    )
    early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min')
    trainer = L.Trainer(
        max_epochs=30,
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1,
        precision='bf16-mixed' if torch.cuda.is_bf16_supported() else 32,
        gradient_clip_val=0.1,
        callbacks=[early_stop],
        logger=False,
        enable_progress_bar=False,
        limit_val_batches=1.0,
        num_sanity_val_steps=0,
    )
    trainer.fit(model, train_loader, val_loader)
    return trainer.callback_metrics['val_loss'].item()

study = optuna.create_study(direction='minimize')
study.optimize(tft_objective, n_trials=10)
best_dropout = study.best_params['dropout']
print('Best dropout:', best_dropout)

# --- Train final model with optimal dropout ---
early_stop = EarlyStopping(monitor='val_loss', patience=6, mode='min')
lr_logger  = LearningRateMonitor(logging_interval='epoch')
checkpoint = ModelCheckpoint(monitor='val_loss', save_top_k=1, mode='min')
logger     = CSVLogger('tft_logs', name='catboost_tft')

precision = 'bf16-mixed' if torch.cuda.is_bf16_supported() else 32
print(f'[Precision] Using {precision}')

tft_model = TemporalFusionTransformer.from_dataset(
    tft_dataset,
    loss=RMSE(),
    log_interval=200,
    reduce_on_plateau_patience=4,
    dropout=best_dropout,
    weight_decay=1e-2,
    **TFT_PARAMS
)

trainer = L.Trainer(
    max_epochs=30,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    precision=precision,
    gradient_clip_val=0.1,
    callbacks=[early_stop, lr_logger, checkpoint],
    logger=logger,
    enable_progress_bar=True,
    limit_val_batches=1.0,
    num_sanity_val_steps=0,
)

trainer.fit(tft_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

if checkpoint.best_model_path:
    tft_model = TemporalFusionTransformer.load_from_checkpoint(checkpoint.best_model_path)
    print('[TFT] best checkpoint:', checkpoint.best_model_path)
else:
    print('[TFT] Warning: no best checkpoint found.')


# Prediction

In [ ]:
# ==============================================================
# Prediction Functions (CatBoost, TFT-mini, XGBoost) -- Modified
# ==============================================================

def convert_to_submission_format(pred_df: pd.DataFrame, sample_df: pd.DataFrame) -> pd.DataFrame:
    out = sample_df.copy()
    wide = pred_df.pivot(index='영업일자', columns='영업장명_메뉴명', values='매출수량')
    for r in range(len(out)):
        date_key = out.at[r, '영업일자']
        if date_key in wide.index:
            for c in out.columns:
                if c == '영업일자':
                    continue
                if c in wide.columns:
                    val = wide.at[date_key, c]
                    out.at[r, c] = 0 if pd.isna(val) else max(float(val), 0.0)
    for c in out.columns:
        if c == '영업일자':
            continue
        out[c] = pd.to_numeric(out[c], errors='coerce').fillna(0).clip(lower=0)
    return out


def prepare_test(df_test_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_test_raw.copy()
    df['영업장명_메뉴명'] = df['영업장명_메뉴명'].astype(str)
    df['업장명'] = df['영업장명_메뉴명'].str.split('_', n=1).str[0]
    df['메뉴명'] = df['영업장명_메뉴명'].str.split('_', n=1).str[1].fillna('NA')
    df['store_id'] = df['업장명'].map(store2id).fillna(-1).astype(int).astype(str)
    df['item_id']  = df['메뉴명'].map(item2id).fillna(-1).astype(int).astype(str)
    df['pair_id']  = (df['업장명'] + '###' + df['메뉴명']).map(pair2id).fillna(-1).astype(int).astype(str)
    df = df.sort_values(['영업장명_메뉴명','영업일자']).reset_index(drop=True)
    df = add_domain_features(df)
    for w in ROLL_WINS:
        df[f'roll_mean_{w}'] = (
            df.groupby('영업장명_메뉴명')['매출수량']
              .transform(lambda s: s.rolling(w, min_periods=1).mean())
        )
    return df

# --- CatBoost prediction ---
def predict_catboost(df_28: pd.DataFrame) -> pd.DataFrame:
    preds=[]
    for pair, sub in df_28.groupby('영업장명_메뉴명'):
        sub=sub.sort_values('영업일자').reset_index(drop=True)
        history=sub.iloc[-ENC_LEN:].copy()
        for h in range(1, PRED_LEN+1):
            fut_date = history['영업일자'].iloc[-1] + timedelta(days=1)
            fut_row=pd.DataFrame([{ '영업일자':fut_date,
                                    '영업장명_메뉴명':pair,
                                    '업장명':history['업장명'].iloc[-1],
                                    '메뉴명':history['메뉴명'].iloc[-1],
                                    'store_id':history['store_id'].iloc[-1],
                                    'item_id':history['item_id'].iloc[-1],
                                    'pair_id':history['pair_id'].iloc[-1],
                                    '매출수량':0 }])
            fut_row=add_domain_features(fut_row)
            tmp_hist=pd.concat([history, fut_row], ignore_index=True)
            for w in ROLL_WINS:
                tmp_hist[f'roll_mean_{w}'] = (
                    tmp_hist.groupby('영업장명_메뉴명')['매출수량']
                           .transform(lambda s: s.rolling(w, min_periods=1).mean())
                )
            fut_row=tmp_hist.iloc[[-1]]
            X=fut_row[cat_features_cols + [f'roll_mean_{w}' for w in ROLL_WINS]].copy()
            fut_feats=fut_row[known_future_cols].copy(); fut_feats.columns=[c+f'_H{h}' for c in fut_feats.columns]
            X=pd.concat([X.reset_index(drop=True), fut_feats.reset_index(drop=True)], axis=1)
            model=cat_models[h]; cat_idx=[i for i,c in enumerate(X.columns) if c in cat_features_cols]
            yhat=float(model.predict(Pool(X, cat_features=cat_idx))[0])
            yhat=max(0.0, yhat)
            preds.append({'영업장명_메뉴명':pair,'h':h,'pred_cat':yhat})
            fut_row.loc[:, '매출수량']=yhat
            history=pd.concat([history, fut_row], ignore_index=True)
    return pd.DataFrame(preds)

# --- XGBoost prediction ---
def predict_xgboost(df_28: pd.DataFrame) -> pd.DataFrame:
    preds=[]
    for pair, sub in df_28.groupby('영업장명_메뉴명'):
        sub=sub.sort_values('영업일자').reset_index(drop=True)
        history=sub.iloc[-ENC_LEN:].copy()
        for h in range(1, PRED_LEN+1):
            fut_date=history['영업일자'].iloc[-1] + timedelta(days=1)
            fut_row=pd.DataFrame([{ '영업일자':fut_date,
                                    '영업장명_메뉴명':pair,
                                    '업장명':history['업장명'].iloc[-1],
                                    '메뉴명':history['메뉴명'].iloc[-1],
                                    'store_id':history['store_id'].iloc[-1],
                                    'item_id':history['item_id'].iloc[-1],
                                    'pair_id':history['pair_id'].iloc[-1],
                                    '매출수량':0 }])
            fut_row=add_domain_features(fut_row)
            tmp_hist=pd.concat([history, fut_row], ignore_index=True)
            for w in ROLL_WINS:
                tmp_hist[f'roll_mean_{w}'] = (
                    tmp_hist.groupby('영업장명_메뉴명')['매출수량']
                           .transform(lambda s: s.rolling(w, min_periods=1).mean())
                )
            fut_row=tmp_hist.iloc[[-1]]
            X=fut_row[cat_features_cols + [f'roll_mean_{w}' for w in ROLL_WINS]].copy()
            fut_feats=fut_row[known_future_cols].copy(); fut_feats.columns=[c+f'_H{h}' for c in fut_feats.columns]
            X=pd.concat([X.reset_index(drop=True), fut_feats.reset_index(drop=True)], axis=1)
            model=xgb_models[h]
            yhat=float(model.predict(X)[0])
            yhat=max(0.0, yhat)
            preds.append({'영업장명_메뉴명':pair,'h':h,'pred_xgb':yhat})
            fut_row.loc[:, '매출수량']=yhat
            history=pd.concat([history, fut_row], ignore_index=True)
    return pd.DataFrame(preds)

# --- TFT prediction ---
def predict_tft(df_28: pd.DataFrame) -> pd.DataFrame:
    tmp=df_28.copy()
    tmp['time_idx']=(tmp['영업일자'] - train['영업일자'].min()).dt.days.astype(int)
    rows=[]
    for pair, sub in tmp.groupby('영업장명_메뉴명'):
        sub=sub.sort_values('영업일자')
        last_date=sub['영업일자'].iloc[-1]
        pid=sub['pair_id'].iloc[-1]; sid=sub['store_id'].iloc[-1]; iid=sub['item_id'].iloc[-1]
        for h in range(1, PRED_LEN+1):
            d=last_date + timedelta(days=h)
            rows.append({'영업장명_메뉴명':pair,'영업일자':d,'pair_id':pid,'store_id':sid,'item_id':iid})
    fut=pd.DataFrame(rows)
    fut=add_domain_features(fut.assign(매출수량=0))
    for w in ROLL_WINS:
        fut[f'roll_mean_{w}']=(
            fut.groupby('영업장명_메뉴명')['매출수량']
               .transform(lambda s: s.rolling(w, min_periods=1).mean())
               .fillna(0)
        )
    fut['time_idx']=(fut['영업일자'] - train['영업일자'].min()).dt.days.astype(int)
    enc_dec=pd.concat([tmp, fut], ignore_index=True)
    enc_dec=enc_dec.fillna(0)
    predict_ds=TimeSeriesDataSet.from_dataset(tft_dataset, enc_dec, predict=True, stop_randomization=True)
    predict_loader=predict_ds.to_dataloader(train=False, batch_size=256, num_workers=2)
    yhat=tft_model.predict(predict_loader, mode='prediction')
    if isinstance(yhat, tuple): yhat=yhat[0]
    fut_sorted=fut.sort_values(['pair_id','영업일자']).reset_index(drop=True)
    series_ids=fut_sorted['pair_id'].unique().tolist()
    out=[]
    for i,pid in enumerate(series_ids):
        pair_name=df_28.loc[df_28['pair_id']==pid, '영업장명_메뉴명'].iloc[0]
        for h in range(1, PRED_LEN+1):
            out.append({'영업장명_메뉴명':pair_name,'h':h,'pred_tft':float(max(0.0, yhat[i, h-1].item()))})
    return pd.DataFrame(out)

# --- helper to get predictions from all models ---
def predict_all_models(df_28: pd.DataFrame) -> pd.DataFrame:
    cat_pred=predict_catboost(df_28)
    tft_pred=predict_tft(df_28)
    xgb_pred=predict_xgboost(df_28)
    pred=cat_pred.merge(tft_pred, on=['영업장명_메뉴명','h'], how='outer')                 .merge(xgb_pred, on=['영업장명_메뉴명','h'], how='outer').fillna(0.0)
    return pred

# --- ensemble prediction using weights ---
def predict_ensemble_for_test_file(test_file_path: str, weights: dict) -> pd.DataFrame:
    df_raw=pd.read_csv(test_file_path, parse_dates=['영업일자'])
    df_28=prepare_test(df_raw)
    pred=predict_all_models(df_28)
    pred['pred_ens'] = (weights['w_cat']*pred.get('pred_cat',0) + weights['w_tft']*pred.get('pred_tft',0) + weights['w_xgb']*pred.get('pred_xgb',0))
    pred['file']=os.path.basename(test_file_path)
    return pred


In [ ]:
# ==============================================================
# Ensemble Weight Optimization (Optuna) -- Added
# ==============================================================

def build_validation_context(train_df):
    contexts=[]
    targets=[]
    for pair, sub in train_df.groupby('영업장명_메뉴명'):
        sub=sub.sort_values('영업일자')
        if len(sub) < ENC_LEN + PRED_LEN:
            continue
        hist=sub.iloc[-(ENC_LEN+PRED_LEN):-PRED_LEN].copy()
        fut=sub.iloc[-PRED_LEN:]['매출수량'].tolist()
        hist['영업장명_메뉴명']=pair
        contexts.append(hist)
        for h,val in enumerate(fut,1):
            targets.append({'영업장명_메뉴명':pair,'h':h,'y_true':float(val)})
    ctx=pd.concat(contexts).reset_index(drop=True)
    tgt=pd.DataFrame(targets)
    return ctx,tgt

val_ctx,val_tgt=build_validation_context(train)
val_pred=predict_all_models(val_ctx).merge(val_tgt, on=['영업장명_메뉴명','h'])
val_pred=val_pred.fillna(0.0)


def smape(y_true, y_pred):
    return np.mean(2*np.abs(y_true - y_pred)/(np.abs(y_true)+np.abs(y_pred)+1e-8))


def weight_objective(trial):
    w_cat=trial.suggest_float('w_cat',0,1)
    w_tft=trial.suggest_float('w_tft',0,1-w_cat)
    w_xgb=1 - w_cat - w_tft
    if w_xgb < 0:
        raise optuna.TrialPruned()
    y_hat=w_cat*val_pred['pred_cat'] + w_tft*val_pred['pred_tft'] + w_xgb*val_pred['pred_xgb']
    return smape(val_pred['y_true'], y_hat)

study_w=optuna.create_study(direction='minimize')
study_w.optimize(weight_objective, n_trials=50)
best_weights=study_w.best_params
best_weights['w_xgb']=1 - best_weights['w_cat'] - best_weights['w_tft']
print('Best weights:', best_weights)


In [ ]:
# -------------------------------
# Prediction 실행
# -------------------------------
all_preds = []
for test_path in sorted(test_paths):
    print(f'[Predict] {test_path}')
    all_preds.append(predict_ensemble_for_test_file(test_path, best_weights))
all_preds = pd.concat(all_preds, ignore_index=True)


# Submission

In [ ]:
# -------------------------------
# Submission 변환 함수 (vectorized)
# -------------------------------
def convert_preds_to_submission(all_preds, sample_submission):
    full_pred_df = pd.concat(all_preds, ignore_index=True)
    full_pred_df['test_id'] = full_pred_df['file'].str.replace('.csv', '', regex=False)
    wide = full_pred_df.pivot_table(index=['test_id','h'], columns='영업장명_메뉴명', values='pred_ens').reset_index()
    wide['영업일자'] = wide['test_id'] + '+' + wide['h'].astype(int).astype(str) + '일'
    wide = wide.drop(columns=['test_id','h'])
    submission = sample_submission.merge(wide, on='영업일자', how='left').fillna(0)
    submission = submission[sample_submission.columns]
    return submission


In [ ]:
# -------------------------------
# Submission 실행
# -------------------------------
sample_submission = pd.read_csv(sample_path)
submission_df = convert_preds_to_submission(all_preds, sample_submission)
output_path = 'submission_ensemble_optimized.csv'
submission_df.to_csv(output_path, index=False)
print('Saved:', output_path)
